# Visualiser des données géographiques avec la librairie Folium

Pouvoir représenter une carte est souvent une composante essentielle d'une analyse de données. 

La librairie **Folium** permet de créer facilement des cartes interactives. Vous pouvez en lire plus sur la documentation officielle - https://www.pypi.python.org/pypi/folium

Dans cette partie, vous verrez : 

- Comment intégrer une carte interactive au sein de l'environnement iPython
- Comment tracer des points à partir d"un DataFrame 
- Comment tracer des zones, en utilisant des repères TopoJSON

## Créer des cartes et des repères

Installez la librairie folium depuis votre terminal grâce à pip : 

In [ ]:
pip install folium

Nous importons tout d'abord la librairie folium ainsi que le module HTML de iPython :

In [ ]:
import folium

Nous définissons les paramètres d'affichage de notre carte en précisant son centre:

In [ ]:
m = folium.Map(location=[48.8655909,2.378983])
m

Nous pouvons ajouter un repère grâce aux méthodes `Marker`, `CircleMarket`: 

In [ ]:
folium.CircleMarker(
    location=[48.89157,2.24099],
    radius=50,
    color='#3186cc',
).add_to(m)

m

In [ ]:
folium.Marker(
    location=[48.89157,2.24099],
).add_to(m)

m

Nous pouvons enfin ajouter du contenu au sein de la pop up des marqueurs que nous avons créé, en passant un chaine de caractère au sein de la méthode **popup**

In [ ]:
folium.Marker(
    location=[48.89157,2.24099],
    popup='Hello!',
).add_to(m)

m

## Créer des repères à partir d'un DataFrame

Nous pouvons donc appliquer la même méthode à partir de données issues d'un DataFrame. 

Nous pouvons par exemple afficher une carte de France des emplacements des restaurants McDonalds. 

Nous importons pandas : 

In [ ]:
import pandas as pd

Nous chargeons le fichier McDonalds_Fr. Celui-ci contient les latitudes, longitudes et nom de tous les McDonalds de France : 

In [ ]:
mcdo = pd.read_csv("data/McDonalds_Fr.csv",encoding='latin-1')
mcdo.head(3)

In [ ]:
mcdo.count()

Comme précédemment, nous déclarons une carte Folium : 

In [ ]:
map_mcdo= folium.Map(location=[48.8655909,2.378983], 
                     zoom_start=5, 
                     tiles='Stamen Toner')

Nous réalisons une boucle à partir des valeurs de notre DataFrame :

In [ ]:
for x in mcdo.values:
    location = []
    location.append(x[1])
    location.append(x[0])
    
    folium.CircleMarker(
        location=location,
        radius=4,
        color='red',
        popup=x[2]
    ).add_to(map_mcdo)
    

Nous appelons enfin la méthode **inline_map** pour afficher notre carte : 

In [ ]:
map_mcdo

## Créer des zones géographiques à partir de pandas et des fichiers GeoJSON

Folium vous permet également de créer vos propres polygônes géographiques en se basant sur les standard TopoJSON / GeoJSON. Comme leur nom l'indique, ces fichiers sont des fichiers JSON décrivant les frontières d'une zone géographique à tracer. 

Il existe de nombreuses ressources en ligne permettant de récupérer des fichiers TopoJSON existant ou de construire les siens. 

Nous prendrons dans cet exemple des fichiers issus de ce repo GitHub - https://github.com/gregoiredavid/france-geojson où sont disponibles les communes, les départements et les régions. 

Nous allons dans cet exemple, réaliser une carte du taux de chômage en France, par département. 

La méthode **GeoJSON** permet d'afficher une zone géographique sur une carte. Il suffit dans ce cas de passer en argument le fichier TopoJSON source : 

In [ ]:
map_departement = folium.Map(location=[48,2], 
                             zoom_start=3)

folium.GeoJson(
    "data/departements.json",
    name='geojson'
).add_to(map_departement)

In [ ]:
map_departement

L'une des fonctionnalités assez sympa de Folium est la possibilité de s'interfacer avec un DataFrame Pandas. Voyez plutôt : 

Nous créons un DataFrame **dep_data** à partir des données de chômage dont nous disposons :

In [ ]:
dep_data = pd.read_csv('data/dep_chomage.csv')

In [ ]:
dep_data.head()

Nous créons ensuite notre objet map_chomage en spécifiant nos paramètres :

In [ ]:
map_chomage = folium.Map(location=[48,2], zoom_start=3)

Nous réalisons ensuite le binding entre les données de notre DataFrame et nos départements. 

Plusieurs éléments sont à noter: 
    
- Nous sélectionnons au sein de **column**, deux colonnes. La première doit présenter une clé commune avec notre fichier TopoJSON
- Le paramètre **key_on** précise la clé à utiliser, au sein du fichier TopoJSON pour lier nos données.  

In [ ]:
folium.Choropleth(
    geo_data="data/departements.json",
    name='choropleth',
    data=dep_data,
    columns=['CODE_DEPT', 'TAUX_CHOMAGE'],
    key_on='feature.properties.code',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Taux de chômage (%)'
).add_to(map_chomage)

In [ ]:
map_chomage